In [1]:
import os
import pandas as pd
import pandas_bokeh
import bokeh
from bokeh.plotting import output_file
from bokeh.models import ColumnDataSource
import numpy as np
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.plotting import figure, show, output_notebook

In [2]:
data_path  = '/data/argoverse_data/v2/train/train_part_1'
# pandas_bokeh.output_notebook()
# pd.set_option('plotting.backend', 'pandas_bokeh')
output_notebook()

Loading BokehJS ...

In [3]:
# 创建示例数据
data = pd.Series([1, 2, 2, 3, 4, 4, 4, 5])

# 获取唯一值
unique_values = data.unique()
print("Unique values:", unique_values)
print("Number of unique values:", len(unique_values))

Unique values: [1 2 3 4 5]
Number of unique values: 5


In [4]:
def get_sub_folders(path):
    # 获取指定目录中的所有子文件夹的名字
    sub_folders = [name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))]
    return sub_folders
 
raw_case_names = get_sub_folders(data_path)
 
 # for raw_file_name in tqdm(self.raw_file_names):
 #            df = pd.read_parquet(os.path.join(self.raw_dir, raw_file_name, f'scenario_{raw_file_name}.parquet'))
 #            map_dir = Path(self.raw_dir) / raw_file_name
 #            map_path = map_dir / sorted(map_dir.glob('log_map_archive_*.json'))[0]
 #            map_data = read_json_file(map_path)
 #            centerlines = {lane_segment['id']: Polyline.from_json_data(lane_segment['centerline'])
 #                           for lane_segment in map_data['lane_segments'].values()}
 #            map_api = ArgoverseStaticMap.from_json(map_path)
 #            data = dict()
 #            data['scenario_id'] = self.get_scenario_id(df)
 #            data['city'] = self.get_city(df)
 #            data['agent'] = self.get_agent_features(df)
 #            data.update(self.get_map_features(map_api, centerlines))

In [5]:
raw_case_name = raw_case_names[0]
data_parquet = os.path.join(data_path,raw_case_name,'scenario_'+ raw_case_name + '.parquet')
raw_case = pd.read_parquet(data_parquet)

In [6]:
def get_case_feature(parquet_path) -> dict:
    num_history_point = 50
    raw_case = pd.read_parquet(data_parquet)
    unique_values = raw_case.start_timestamp.unique()

    feature_dict = {}
    
    # for time in unique_values:
    #     print(f"{time:f}"," ")
    # print("Number of unique values:", len(unique_values))
    
    end_unique_values = raw_case.end_timestamp.unique()
    
    # for time in end_unique_values:
    #     print(f"{time:f}"," ")
    # print("Number of unique values:", len(end_unique_values))
    
    # print("time horizen is: " , (end_unique_values[0] - unique_values[0])* 1e-9, "s")
    # print("time step is: " , (end_unique_values[0] - unique_values[0])* 1e-9/(raw_case.num_timestamps[0] -1), "s")
    track_id_unique_values = raw_case.track_id.unique()
    trak_id_sum = len(track_id_unique_values) -1
    valid_list = []
    valid_agent_sum = 0
    for trak_id, track_df in raw_case.groupby("track_id"):
        # print(track_df)
        count_valid = (track_df['timestep'] < 50).sum()
        valid_list.append(float(count_valid)/float(num_history_point))
        if float(count_valid)/float(num_history_point) > 0.0:
            valid_agent_sum = valid_agent_sum + 1
        # print("valid rate", f"{float(count_valid)/float(num_history_point)}f")
    # print("Number of unique values:", len(track_id_unique_values))

    # get AV vel feature
    filtered_rows = raw_case[raw_case['track_id'] == 'AV']

    # previous_position = None
    # distances = []
    # vel = []
    # for index, row in filtered_rows.iterrows():
    #     current_position = np.array([row['position_x'], row['position_y']])
        
    #     if previous_position is not None:
    #         distance = np.linalg.norm(current_position - previous_position)
    #         distances.append(distance)
    #         vel.append((row['velocity_x'] **2 + row['velocity_y']** 2) ** 0.5 )
    #     else:
    #         distances.append(0)  # 第一行距离为0
    #         vel.append(0)
        
    #     previous_position = current_position
    # for n in range(min(len(vel),len(distances))):
    #     print("dis:", distances[n],  " vel:", vel[n] * 0.1)  
    
    vel_list = []

    for index, row in filtered_rows.iterrows():
        current_v = np.array([row['velocity_x'], row['velocity_y']])
        vel_list.append(np.linalg.norm(current_v))
    
    max_vel = max(vel_list)
    min_vel = min(vel_list)
    aver_vel = sum(vel_list) / len(vel_list)

    feature_dict = {    
        "AV_max_vel":max_vel,
        "AV_min_vel":min_vel,
        "AV_average_vel":aver_vel,
        "valid_list":valid_list,
        "valid_agent_sum":valid_agent_sum
    }
    
    return feature_dict

In [7]:
df_feature_all = None

for raw_case_name in raw_case_names:
    data_parquet = os.path.join(data_path,raw_case_name,'scenario_'+ raw_case_name + '.parquet')
    data_feature = get_case_feature(data_parquet)
    if df_feature_all is None:
        df_feature_all = pd.DataFrame([data_feature])
    else:
        df_feature_all = pd.concat([df_feature_all,pd.DataFrame([data_feature])], ignore_index = False)
df_feature_all

,AV_max_vel,AV_min_vel,AV_average_vel,valid_list,valid_agent_sum
0,1.507082,0.002490,0.527935,"[1.0, 1.0, 1.0, 0.6, 0.96, 0.7, 1.0, 0.96, 1.0...",56
0,9.806533,0.000384,3.987523,"[0.22, 0.42, 0.98, 0.8, 0.84, 0.84, 1.0, 0.46,...",33
0,13.323682,4.380413,11.953260,"[1.0, 1.0, 1.0, 1.0, 0.3, 0.94, 0.78, 0.54, 0....",12
0,9.739502,0.000000,6.518794,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.28, 1.0, 1.0,...",47
0,12.164614,0.007204,10.024606,"[0.92, 1.0, 1.0, 0.98, 1.0, 1.0, 0.68, 1.0, 1....",36
...,...,...,...,...,...
0,11.513977,0.029528,10.375436,"[1.0, 1.0, 0.36, 1.0, 0.68, 0.6, 1.0, 0.5, 1.0...",45
0,9.274381,0.000021,2.346213,"[1.0, 0.38, 0.94, 1.0, 1.0, 0.88, 1.0, 1.0, 1....",15
0,8.179656,0.026141,4.829925,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",21
0,11.060655,4.296875,10.535135,"[1.0, 1.0, 1.0, 0.38, 0.88, 0.82, 0.24, 0.56, ...",24


In [8]:

def generate_bar_data(match_pts,data):
    cls = []
    cls_sum = []
    pre_pt = None
    for pts in match_pts:
        if pre_pt is None:
            pre_pt = pts
            continue
        else:
            cls.append(f"{pre_pt:.1f}" + " - " + f"{pts:.1f}")
            condition = (data >= pre_pt) & (data < pts)
            cls_sum.append(condition.sum())
            pre_pt = pts
    return {"cls": cls, "cls_sum": cls_sum}


vel_check_points = [0.0,1.0,10.0,20.0,30.0,60.0]
AV_max_vel_cls_res = generate_bar_data(vel_check_points,df_feature_all['AV_max_vel'])

# AV_min_vel_points = [0.0,1.0,10.0,20.0,30.0,60.0]
# AV_min_vel_cls_res = generate_bar_data(vel_check_points,df_feature_all['AV_min_vel'])

# AV_ave_vel_points = [0.0,1.0,10.0,20.0,30.0,60.0]
# AV_ave_vel_cls_res = generate_bar_data(vel_check_points,df_feature_all['AV_average_vel'])

feature_vel_list = ['AV_max_vel',"AV_min_vel","AV_average_vel"]

feat_vel_data = {}
for feature in feature_vel_list:
    res = generate_bar_data(vel_check_points,df_feature_all[feature])
    count = 0
    for ckp_feature in res['cls']:
        if ckp_feature not in feat_vel_data.keys():
            feat_vel_data[ckp_feature] = [res['cls_sum'][count]]
        else:
            feat_vel_data[ckp_feature].append(res['cls_sum'][count])
        count = count + 1
    
print(feat_vel_data)
# print(AV_max_vel_cls_res['cls_sum'])
# print(AV_min_vel_cls_res['cls_sum'])
# print(AV_ave_vel_cls_res['cls_sum'])


x = [ (feature, ckp_feature) for feature in feature_vel_list for ckp_feature in AV_max_vel_cls_res['cls'] ]
counts = sum(zip(*feat_vel_data.values()),())

source = ColumnDataSource(data=dict(x=x, counts=counts))

p = figure(x_range=FactorRange(*x), height=350, title="Fruit Counts by Year",
           toolbar_location=None, tools="")

p.vbar(x='x', top='counts', width=0.9, source=source)

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

show(p)
# show(p)

{'0.0 - 1.0': [3211, 16259, 4926], '1.0 - 10.0': [9140, 3583, 11419], '10.0 - 20.0': [7640, 149, 3646], '20.0 - 30.0': [0, 0, 0], '30.0 - 60.0': [0, 0, 0]}


In [9]:
# demo

fruits = ['Apples', 'Pears', 'Nectarines', 'Plums', 'Grapes', 'Strawberries']
years = ['2015', '2016', '2017']

data = {'fruits' : fruits,
        '2015'   : [2, 1, 4, 3, 2, 4],
        '2016'   : [5, 3, 3, 2, 4, 6],
        '2017'   : [3, 2, 4, 4, 5, 3]}

# this creates [ ("Apples", "2015"), ("Apples", "2016"), ("Apples", "2017"), ("Pears", "2015), ... ]
x = [ (fruit, year) for fruit in fruits for year in years ]
print(x)
counts = sum(zip(data['2015'], data['2016'], data['2017']), ()) # like an hstack

source = ColumnDataSource(data=dict(x=x, counts=counts))

p = figure(x_range=FactorRange(*x), height=350, title="Fruit Counts by Year",
           toolbar_location=None, tools="")

p.vbar(x='x', top='counts', width=0.9, source=source)

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

show(p)

[('Apples', '2015'), ('Apples', '2016'), ('Apples', '2017'), ('Pears', '2015'), ('Pears', '2016'), ('Pears', '2017'), ('Nectarines', '2015'), ('Nectarines', '2016'), ('Nectarines', '2017'), ('Plums', '2015'), ('Plums', '2016'), ('Plums', '2017'), ('Grapes', '2015'), ('Grapes', '2016'), ('Grapes', '2017'), ('Strawberries', '2015'), ('Strawberries', '2016'), ('Strawberries', '2017')]
